In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *

In [2]:
#Starting session
config = pyspark.SparkConf().setAll([('spark.executor.memory', '8g')])
spark = SparkSession.builder.appName('BigData1').getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

In [3]:
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

In [7]:
joined = spark.read.parquet("G:\\Master\\Big Data\\Data\\D.C\\New\\joined_extended.parquet")

In [5]:
joined

time,duration,start_date,end_date,start_station_id,start_station_name,end_station_id,end_station_name,bike_number,member_type,lat_start,long_start,lat_end,long_end,abs_distance,date_time,air_temp,wind_speed,weather_summary,weather_condition,visibility,weather_class
2019-10-31 09:51:09,251,2019-10-31 09:51:09,2019-10-31 09:55:20,31641,2nd St & Massachu...,31655,New Jersey Ave & ...,W21832,Member,38.894972,-77.003135,38.897108,-77.011616,771,2019-10-31 09:19:00,19.0,13.8,overcast,null,10.0,cloudy
2019-10-31 09:51:30,213,2019-10-31 09:51:30,2019-10-31 09:55:04,31612,D St & Maryland A...,31623,Columbus Circle /...,W23251,Member,38.894758,-76.997114,38.89696,-77.00493,719,2019-10-31 09:19:00,19.0,13.8,overcast,null,10.0,cloudy
2019-10-31 09:51:30,186,2019-10-31 09:51:30,2019-10-31 09:54:37,31610,Eastern Market / ...,31605,3rd & D St SE,W01026,Member,38.886952,-76.996806,38.8851,-77.0023,518,2019-10-31 09:19:00,19.0,13.8,overcast,null,10.0,cloudy
2019-10-31 09:51:39,296,2019-10-31 09:51:39,2019-10-31 09:56:36,31255,24th & N St NW,31293,31st & Water St NW,W23751,Member,38.9066,-77.05152,38.902814,-77.061043,925,2019-10-31 09:19:00,19.0,13.8,overcast,null,10.0,cloudy
2019-10-31 09:51:45,121,2019-10-31 09:51:45,2019-10-31 09:53:47,31263,10th & K St NW,31227,13th St & New Yor...,W23787,Member,38.9024,-77.02622,38.900283,-77.029822,391,2019-10-31 09:19:00,19.0,13.8,overcast,null,10.0,cloudy
2019-10-31 09:52:00,1119,2019-10-31 09:52:00,2019-10-31 10:10:39,31529,Edgewood Rec Center,31244,4th & E St SW,W22220,Member,38.924144,-77.002098,38.88412,-77.017445,4644,2019-10-31 09:19:00,19.0,13.8,overcast,null,10.0,cloudy
2019-10-31 09:52:00,802,2019-10-31 09:52:00,2019-10-31 10:05:23,31124,14th & Irving St NW,31221,18th & M St NW,W00563,Member,38.928552,-77.032224,38.90506687455313,-77.04177886247635,2739,2019-10-31 09:19:00,19.0,13.8,overcast,null,10.0,cloudy
2019-10-31 09:52:00,465,2019-10-31 09:52:00,2019-10-31 09:59:46,31285,22nd & P ST NW,31257,22nd & I St NW / ...,W20321,Member,38.909394,-77.048728,38.90088,-77.048911,947,2019-10-31 09:19:00,19.0,13.8,overcast,null,10.0,cloudy
2019-10-31 09:52:01,303,2019-10-31 09:52:01,2019-10-31 09:57:04,31127,22nd & H St NW,31293,31st & Water St NW,W22289,Member,38.89892546716842,-77.04885238395946,38.902814,-77.061043,1140,2019-10-31 09:19:00,19.0,13.8,overcast,null,10.0,cloudy
2019-10-31 09:52:10,621,2019-10-31 09:52:10,2019-10-31 10:02:31,31636,New Jersey Ave & ...,31232,7th & F St NW / N...,W20672,Member,38.90733256111448,-77.01536044478416,38.89728304853854,-77.02219069004059,1264,2019-10-31 09:19:00,19.0,13.8,overcast,null,10.0,cloudy


In [8]:
from pyspark.sql.functions import dayofyear, month, hour, sum, avg, count

joined = joined.withColumn('dayofyear', dayofyear(joined["time"]))
joined = joined.withColumn('hour', hour(joined["time"]))
joined = joined.withColumn('month', month(joined["time"]))

In [9]:
@F.udf
def mode(x):
    from collections import Counter
    return Counter(x).most_common(1)[0][0]

In [8]:
from pyspark.sql.functions import col, collect_list
from pyspark.sql.functions import dayofyear, month, sum, avg, count

generated = joined.groupBy(col('hour'), col('dayofyear'), col('month'), col('start_station_id')).agg(sum("duration").alias("total_duration"), count('time').alias('count'), avg('air_temp').alias('air_temp'), avg('visibility').alias('visibility'), avg('wind_speed').alias('wind_speed'), avg('abs_distance').alias('abs_distance'), mode(collect_list('weather_class')).alias('weather_class'), avg('lat_start').alias('lat'), avg('long_start').alias('long'))

In [9]:
generated

hour,dayofyear,month,start_station_id,total_duration,count,air_temp,visibility,wind_speed,abs_distance,weather_class,lat,long
0,24,1,31270,409,1,7.6000000000000005,10.0,0.0,1058.0,cloudy,38.894851,-77.02324
0,36,2,31283,1033,1,8.900000000000002,10.0,0.0,2403.0,clear,38.903658,-77.031737
0,39,2,31241,426,1,10.0,10.0,3.44,1391.0,cloudy,38.9059,-77.0325
0,41,2,31408,597,1,-1.2000000000000002,10.0,8.05,1574.0,clear,38.975219,-77.016855
0,51,2,31503,867,1,1.6000000000000014,10.0,4.61,1868.0,clear,38.9126,-77.0135
0,61,3,31201,367,1,0.9999999999999984,10.0,9.22,921.0,light rain,38.90985,-77.034438
0,68,3,31261,4932,2,0.29999999999999954,3.0,0.0,3069.0,snowy,38.892459,-77.046567
0,73,3,31251,667,1,-2.5999999999999996,10.0,0.0,0.0,clear,38.903819,-77.0284
0,75,3,31299,1505,2,20.4,10.0,18.41,2000.0,clear,38.912644,-77.04564
0,90,3,31121,234,1,20.700000000000003,10.0,11.5,650.0,cloudy,38.923583,-77.050046


In [11]:
generated.select(F.max("count"))

max(count)
101


In [26]:
generated.count()

1277508

In [12]:
generated.write.parquet("G:\\Master\\Big Data\\Data\\D.C\\New\\ml_dataset_extended_final.parquet")

In [4]:
generated = spark.read.parquet("G:\\Master\\Big Data\\Data\\D.C\\New\\ml_dataset_extended_final.parquet")

In [ ]:
#fill with zeros where neccessary - missing stations - find distinct stations

In [5]:
distinct_stations = generated.select('start_station_id').distinct()

In [6]:
distinct_stations.count()

579

In [11]:
from pyspark.sql.functions import col, collect_list
from pyspark.sql.functions import dayofyear, month, sum, avg, count

the_base_distinct = joined.groupBy(col('hour'), col('dayofyear'), col('month')).agg(avg('air_temp').alias('air_temp'), avg('visibility').alias('visibility'), avg('wind_speed').alias('wind_speed'), mode(collect_list('weather_class')).alias('weather_class'))

In [11]:
the_base_distinct.cache()

hour,dayofyear,month,air_temp,visibility,wind_speed,weather_class
5,346,12,-0.6555555555555549,10.0,6.262222222222224,clear
11,143,5,16.225619834710777,10.0,8.34938016528931,clear
11,308,11,0.4163538873994616,10.0,0.0,foggy
13,306,11,3.713470115967837,10.0,0.0,foggy
17,346,12,2.425912838633716,10.0,4.31135453474679,clear
22,142,5,21.064768683274067,10.0,8.049999999999988,clear
1,191,7,23.57435897435899,10.0,5.690769230769231,clear
1,268,9,21.327777777777783,10.0,5.430555555555555,clear
5,12,1,-3.6999999999999997,10.0,3.4400000000000004,clear
6,84,3,7.708813559322077,10.0,5.2432203389830505,cloudy


In [12]:
the_base_distinct.count()

8746

In [13]:
stations_list = distinct_stations.toPandas()

In [20]:
stations_list.to_pickle("G:\\Master\\Big Data\\Data\\D.C\\New\\stations.pickle")

In [13]:
import pandas as pd

stations_list = pd.read_pickle("G:\\Master\\Big Data\\Data\\D.C\\New\\stations.pickle")

In [14]:
stations_list = list(stations_list["start_station_id"])

In [15]:
the_base_distinct = the_base_distinct.withColumn("station_list", F.array([F.lit(x) for x in stations_list]))

In [16]:
new_base = the_base_distinct.select("hour", "dayofyear", "month", "air_temp", "visibility", "wind_speed", "weather_class", F.explode("station_list").alias("station"))

In [20]:
new_base

hour,dayofyear,month,air_temp,visibility,wind_speed,weather_class,station
5,346,12,-0.6555555555555549,10.0,6.262222222222224,clear,31118
5,346,12,-0.6555555555555549,10.0,6.262222222222224,clear,31518
5,346,12,-0.6555555555555549,10.0,6.262222222222224,clear,31039
5,346,12,-0.6555555555555549,10.0,6.262222222222224,clear,31713
5,346,12,-0.6555555555555549,10.0,6.262222222222224,clear,31067
5,346,12,-0.6555555555555549,10.0,6.262222222222224,clear,31418
5,346,12,-0.6555555555555549,10.0,6.262222222222224,clear,31217
5,346,12,-0.6555555555555549,10.0,6.262222222222224,clear,31290
5,346,12,-0.6555555555555549,10.0,6.262222222222224,clear,32052
5,346,12,-0.6555555555555549,10.0,6.262222222222224,clear,31205


In [17]:
new_base = new_base.withColumn("count", F.lit(0))

In [22]:
new_base

hour,dayofyear,month,air_temp,visibility,wind_speed,weather_class,station,count
5,346,12,-0.6555555555555549,10.0,6.262222222222224,clear,31118,0
5,346,12,-0.6555555555555549,10.0,6.262222222222224,clear,31518,0
5,346,12,-0.6555555555555549,10.0,6.262222222222224,clear,31039,0
5,346,12,-0.6555555555555549,10.0,6.262222222222224,clear,31713,0
5,346,12,-0.6555555555555549,10.0,6.262222222222224,clear,31067,0
5,346,12,-0.6555555555555549,10.0,6.262222222222224,clear,31418,0
5,346,12,-0.6555555555555549,10.0,6.262222222222224,clear,31217,0
5,346,12,-0.6555555555555549,10.0,6.262222222222224,clear,31290,0
5,346,12,-0.6555555555555549,10.0,6.262222222222224,clear,32052,0
5,346,12,-0.6555555555555549,10.0,6.262222222222224,clear,31205,0


In [25]:
generated = generated.withColumnRenamed("start_station_id", "station")

In [23]:
new_base.count()

5063934

In [19]:
new_base.write.parquet("G:\\Master\\Big Data\\Data\\D.C\\New\\HELP\\empty_base.parquet")

In [26]:
base = generated.select("hour", "dayofyear", "month", "air_temp", "visibility", "wind_speed", "weather_class", "count", "station").union(new_base)

In [37]:
base.count()

Py4JJavaError: An error occurred while calling o967.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 136.0 failed 1 times, most recent failure: Lost task 3.0 in stage 136.0 (TID 10703, localhost, executor driver): java.io.FileNotFoundException: C:\Users\IrenaDJ\AppData\Local\Temp\blockmgr-4f297151-eb73-4d20-b9f7-1103452cbb9b\2f\temp_shuffle_c9d886ac-2607-444c-ab57-a7550f6c4be1 (The system cannot find the path specified)
	at java.io.FileOutputStream.open0(Native Method)
	at java.io.FileOutputStream.open(FileOutputStream.java:270)
	at java.io.FileOutputStream.<init>(FileOutputStream.java:213)
	at org.apache.spark.storage.DiskBlockObjectWriter.initialize(DiskBlockObjectWriter.scala:103)
	at org.apache.spark.storage.DiskBlockObjectWriter.open(DiskBlockObjectWriter.scala:116)
	at org.apache.spark.storage.DiskBlockObjectWriter.write(DiskBlockObjectWriter.scala:237)
	at org.apache.spark.shuffle.sort.BypassMergeSortShuffleWriter.write(BypassMer

In [29]:
final = base.orderBy(base['count'], ascending = False)

In [32]:
final = final \
    .groupBy("hour", "dayofyear", "month", "station") \
    .agg(F.first("count").alias("count"), F.first("air_temp").alias("air_temp"), F.first("visibility").alias("visibility"), \
    F.first("wind_speed").alias("wind_speed"), \
    F.first("weather_class").alias("weather_class"))

In [35]:
final.write.parquet("G:\\Master\\Big Data\\Data\\D.C\\New\\grand_dataset.parquet")

In [36]:
final.count()

1286254